In [1]:
!pip install pandas
!pip install pm4py

In [2]:
import pandas as pd
import pm4py

# Adapting the txt file to the task at hand
The dataset given is a file txt from which we will take only the data we need and make save in another file

In [3]:
# path of the correct dataset given
path = 'dataset/'

# file open
f = open(path + 'mpo_def18-gen.txt', 'r')
lines = f.readlines()[:]

In [4]:
lines = [line.replace('\n','') for line in lines]
lines[:5]

['entry(1,begin_of_process,monday,monday6637,none,none).',
 'entry(2,begin_of_activity,monday,monday6637,sit_down,1).',
 'entry(3,begin_of_activity,monday,monday6637,eat_meal,1).',
 'entry(4,end_of_activity,monday,monday6637,eat_meal,1).',
 'entry(5,end_of_activity,monday,monday6637,sit_down,1).']

In [5]:
# get the relevant information of all lines
lines = [line.split('(')[1] for line in lines]
lines = [line.split(')')[0] for line in lines]
lines[:5]

['1,begin_of_process,monday,monday6637,none,none',
 '2,begin_of_activity,monday,monday6637,sit_down,1',
 '3,begin_of_activity,monday,monday6637,eat_meal,1',
 '4,end_of_activity,monday,monday6637,eat_meal,1',
 '5,end_of_activity,monday,monday6637,sit_down,1']

In [6]:
# Save the CSV
write_file = open(path + 'data/def18-gen_data.txt', 'w')
write_file.write("\n".join(lines))

25022

# Import data as CSV

In [7]:
# According to the paper, we set the column name as the description of the 
# 6-tuple
columns_name = [
    'EventTimestamp' , 
    'TypeOfEvent', 
    'ReferenceWorkflow',
    'CaseIdentifier',
    'NameActivity',
    'OccurrencyOfActivity'
]

df = pd.read_csv(path + 'data/def30_data.txt', names = columns_name)
df.head()

,EventTimestamp,TypeOfEvent,ReferenceWorkflow,CaseIdentifier,NameActivity,OccurrencyOfActivity
0,1,begin_of_process,monday,monday6637,none,none
1,2,begin_of_activity,monday,monday6637,sit_down,1
2,3,begin_of_activity,monday,monday6637,eat_meal,1
3,4,end_of_activity,monday,monday6637,eat_meal,1
4,5,end_of_activity,monday,monday6637,sit_down,1


In [8]:
# For doing process mining, we need a timestamp, in this case we convert the EventTimestamp

# CONTROLL THIS
df['EventTimestamp'] = pd.to_datetime(df['EventTimestamp'])
df.head()

,EventTimestamp,TypeOfEvent,ReferenceWorkflow,CaseIdentifier,NameActivity,OccurrencyOfActivity
0,1970-01-01 00:00:00.000000001,begin_of_process,monday,monday6637,none,none
1,1970-01-01 00:00:00.000000002,begin_of_activity,monday,monday6637,sit_down,1
2,1970-01-01 00:00:00.000000003,begin_of_activity,monday,monday6637,eat_meal,1
3,1970-01-01 00:00:00.000000004,end_of_activity,monday,monday6637,eat_meal,1
4,1970-01-01 00:00:00.000000005,end_of_activity,monday,monday6637,sit_down,1


In [9]:
log = pm4py.format_dataframe(df,
                            case_id='TypeOfEvent',
                            activity_key='NameActivity', 
                            timestamp_key='EventTimestamp'
                           ) 

log = pm4py.convert_to_event_log(log)
pm4py.write_xes(log, path + 'data/def30_data.xes')

/Users/vitolosavio/anaconda3/lib/python3.11/site-packages/pm4py/utils.py:538: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")


exporting log, completed traces ::   0%|          | 0/5 [00:00<?, ?it/s]